In [121]:
# Imports
import networkx as nx
import numpy as np
import cvxpy as cp

# Helper Functions

def getEdgeExplanation(oldEdge, newEdge):
    
    if oldEdge != newEdge:
    
        if oldEdge['noWay'] != newEdge['noWay']:
            exp = "is only one way (on the other side). If it was a two way edge,"
            
        elif oldEdge['isClosed'] != newEdge['isClosed']:
            exp ="is closed. If it was open,"
            
        elif oldEdge['maxSpeed'] != newEdge['maxSpeed']:
            exp = "has a maximim speed of {}. If it had a maximum speed of {},".format(oldEdge['maxSpeed'], newEdge['maxSpeed'])
            
        elif oldEdge['speed'] != newEdge['speed']:
            exp = "has a current speed of {} (heavy traffic). If it had a current speed of {} (less traffic),".format(oldEdge['speed'], newEdge['speed'])
            
        exp += ' it would help make the path optimal.'
        
    else:
        # Default explanation, no change.
        exp = 'does not require any changes'
        
    return exp

def getGraphExplanation(oldGraph, newGraph, path):
    explanations = {}
    
    for i in range(len(path) - 1):
        j = i + 1
        source = path[i]
        target = path[j]
        
        oldEdge = oldGraph.get_edge_data(source, target)[0]
        newEdge = newGraph.get_edge_data(source, target)[0]
        
        exp = getEdgeExplanation(oldEdge, newEdge)
        
        explanations[source, target] = "Edge ({}, {}) ".format(source, target) + exp
        
    return explanations

def explanationsPrinter(explanations):
    
    print("These are the explanations why the desired path is nont the optimal path, and how it would be an optimal path:")
    
    for edge in explanations:
        print( "   -   " + explanations[edge] )

def calculateWeight(data):
    inf = 1e6
    
    if data['speedOrMaxSpeed'] == 1:
        weight = getInverse(data['speed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
    else:
        weight = getInverse(data['maxSpeed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
        
    return weight

def updateEdgeWeight(graph, source, target, data):
    graph[source][target][0]['weight'] = calculateWeight(data)

def updateGraphWeights(graph):
    for (i, j, data) in graph.edges(data=True):
        updateEdgeWeight(graph, i, j, data)
    
def getPathWeight(path, graph):
    weights = 0
    for i in range(len(path) - 1):
        j = i + 1
        source = path[i]
        target = path[j]
        weights += graph[source][target][0]['weight']
        
    return weights
        
def getInverse(speed):
    return 1/speed

def addReverseEdges(graph):
    for (i, j) in graph.edges():
        if (j, i) not in graph.edges():
            graph.add_edge(
                            j, i, 
                            weight = np.nan,
                            noWay = 1,
                            isClosed = 0,
                            length = 0,
                            speed = graph[i][j][0]['speed'],
                            maxSpeed = graph[i][j][0]['maxSpeed'],
                            speedOrMaxSpeed = 1
                        )
            updateEdgeWeight(graph, j, i, graph[j][i][0])
    
def cleanGraphAttributes(graph):
    for (i, j) in graph.edges():
        if graph[i][j][0]['isClosed'] == 1:
            graph[i][j][0]['length'] = 0
    
def prepareGraph(graph):
    cleanGraphAttributes(G)
    updateGraphWeights(G)
    addReverseEdges(graph)
    
    
def inverseShortestPath(graph, desiredPath):
    # Constants
    inf = 1e6
    epsilon = 1e-6
    possibleMaxSpeeds = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    inversePossibleMaxSpeeds = [getInverse(s) for s in possibleMaxSpeeds] + [0]
    inversePossibleMaxSpeeds = np.asarray(inversePossibleMaxSpeeds)
    
    # Some graph and path data
    n = len(graph.nodes())
    m = len(graph.edges())
    source = desiredPath[0]
    target = desiredPath[len(desiredPath) - 1]

    # Original Graph data
    noWay = []
    areClosed = []
    inverseSpeeds = []
    inverseMaxSpeeds = []
    lengths = []
    maxSpeeds_H1E = []
    speedOrMaxSpeed_original = []
    
    # Edges data, and their indecies
    edges = []
    edgeIndex = {}
    for (i, j, data) in graph.edges(data=True):
        # Edges Index
        edgeIndex[i,j] = len(edges)
        # Add edge
        edges.append([i, j])
        
        # Data:
        noWay.append(data['noWay'])
        areClosed.append(data['isClosed'])
        inverseSpeeds.append(getInverse(data['speed']))
        inverseMaxSpeeds.append(getInverse(data['maxSpeed']))
        lengths.append(data['length'])
        speedOrMaxSpeed_original.append(data['speedOrMaxSpeed'])
        
        # hot 1 encoding original data
        hot1E = []
        for ms in inversePossibleMaxSpeeds:
            if getInverse(data['maxSpeed']) == ms:
                hot1E.append(1)
            else:
                hot1E.append(0)
        maxSpeeds_H1E.append(hot1E)
        
        
        
    # Nodes data, and their indecies
    nodeIndex = {}
    nodes = []
    for n in graph.nodes:
        # Node Index
        nodeIndex[n] = len(nodes)
        # Add Node
        nodes.append(n)
        
        
    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))

    for i in range(len(nodes)):
        for neighbour in graph.adj[nodes[i]]:
            # Filling A
            j = edgeIndex[nodes[i], neighbour]
            A[i,j] = 1
            if (neighbour, nodes[i]) in edgeIndex:
                j = edgeIndex[neighbour, nodes[i]]
                A[i,j] =-1
            
        # Filling b
        if nodes[i] == source:
            b[i] = 1
        if nodes[i] == target:
            b[i] = -1
            
            
    # optimal x
    path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    xstar = np.zeros(len(edges))
    for p in range(len(path)-1):
        j = edgeIndex[path[p], path[p+1]]
        xstar[j] = 1

    # desired x
    xzero = np.zeros(len(edges))
    for p in range(len(desiredPath)-1):
        j = edgeIndex[desiredPath[p], desiredPath[p+1]]
        xzero[j] = 1
        


    # LP (ISP):
    
    # Original Data as numpy array. (for mathematical applications)
    noWay_original = np.asarray(noWay)
    areClosed_original = np.asarray(areClosed)
    inverseSpeeds_original = np.asarray(inverseSpeeds)
    inverseMaxSpeeds_original = np.asarray(inverseMaxSpeeds)
    maxSpeeds_H1E_original = np.asarray(maxSpeeds_H1E)
    speedOrMaxSpeed_original = np.asarray(speedOrMaxSpeed_original)
    
    
    # Variables
    pi_ = cp.Variable(len(nodes)) #(2d)
    lambda_ = cp.Variable(len(edges)) #(2d)
    
    noWay_ = cp.Variable(len(edges), boolean=True)
    areClosed_ = cp.Variable(len(edges), boolean=True)
    inverseSpeeds_ = cp.Variable(len(inverseSpeeds_original))
    inverseMaxSpeeds_ = cp.Variable(len(edges))
    maxSpeeds_H1E_ = cp.Variable(maxSpeeds_H1E_original.shape, boolean=True)
    # If 1, then change speed, else if 0 change maxSpeed.
    speedOrMaxSpeed_ = cp.Variable(len(edges), boolean=True)
        
        
    # Constraints
    constraints = []
    
    #(2e)
    for j in range(len(edges)):
        if xzero[j] == 0:
            constraints.append( lambda_[j] >= 0 )
            
            
    # Hot 1 Encoding    
    for row in maxSpeeds_H1E_:
        constraints.append( sum(row) == 1)
        
    
    for j in range(len(edges)):
        
        # Hot 1 Encoding, For all edges in G
        constraints.append( inverseMaxSpeeds_[j] == inversePossibleMaxSpeeds.T @ maxSpeeds_H1E_[j] )
        
        
        if xzero[j] == 1: #for all j in desired path
            d_j = inverseSpeeds_[j] * lengths[j] + inverseMaxSpeeds_[j] * lengths[j] + inf * noWay_[j] + inf * areClosed_[j]
        
            # sum_i a_ij * pi_i = d_j,               (2b)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
            
            # inverseSpeed is at least the max speed; ie speed is at most max speed.
            #constraints.append( inverseSpeeds_[j] >= inverseMaxSpeeds_original[j] )
            
            # if speed/maxSpeed >= 3/4 choose the maxSpeed to change, else the speed.
            if inverseMaxSpeeds_original[j] / inverseSpeeds_original[j] >= 3/4:
                # Change maxSpeed
                constraints.append( speedOrMaxSpeed_[j] == 0 )
                constraints.append( inverseMaxSpeeds_[j] >= epsilon )
            else:
                # Change speed
                constraints.append( speedOrMaxSpeed_[j] == 1 )
                
                
            # Lower bound and Upper bounds are 0, if we use the other metric. For speed and max Speed.
            constraints.append( (1 - speedOrMaxSpeed_[j]) * min(inversePossibleMaxSpeeds) <= inverseMaxSpeeds_[j] )
            constraints.append( (1 - speedOrMaxSpeed_[j]) * max(inversePossibleMaxSpeeds) >= inverseMaxSpeeds_[j] )
            
            constraints.append( speedOrMaxSpeed_[j] * inverseMaxSpeeds_original[j] <= inverseSpeeds_[j] )
            constraints.append( speedOrMaxSpeed_[j] * inverseSpeeds_original[j] >= inverseSpeeds_[j] )
            
        else: # for all j not in desired path
            d_j = inverseSpeeds_[j] * lengths[j] + inf * noWay_[j] + inf * areClosed_[j]
            
            # sum_i a_ij * pi_i + lambda_j = d_j,    (2c)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
            
            # Do not change datta
            constraints.append( noWay_[j] == noWay_original[j] )
            constraints.append( areClosed_[j] == areClosed_original[j] )
            constraints.append( inverseSpeeds_[j] == inverseSpeeds_original[j] )
            constraints.append( inverseMaxSpeeds_[j] == inverseMaxSpeeds_original[j] )
            constraints.append( maxSpeeds_H1E_[j] == maxSpeeds_H1E_original[j] )
            constraints.append( speedOrMaxSpeed_[j] == speedOrMaxSpeed_original[j] )
            
            # Keep using speed, and not maxSpeed
            constraints.append( speedOrMaxSpeed_[j] == 1)
    
    penalty1 = 5
    penalty2 = 100
    # Cost function, split up
    cost1 = cp.norm1(cp.multiply(inverseSpeeds_ - inverseSpeeds_original, penalty1))
    cost2 = cp.norm1(cp.multiply(inverseMaxSpeeds_ - inverseMaxSpeeds_original, penalty2))
    cost3 = cp.norm1(noWay_ - noWay_original)
    cost4 = cp.norm1(areClosed_ - areClosed_original)
            
            
    # Final Cost funnction
    cost = cost1 + cost2 + cost3 + cost4
    
        
    # Forming the problem
    prob = cp.Problem(cp.Minimize(cost), constraints)
    
    # Solve the problem
    #prob.solve(solver=cp.GUROBI, verbose=True) #Detailed
    prob.solve(solver=cp.GUROBI) # using gurobi
    print("\nThe optimal value is", prob.value)
    
    #Helper print statements
    # print('original speedInv: ', inverseSpeeds_original)
    # print('optimal speedInv: ', inverseSpeeds_.value)
    # print('\n')
    # print('original speedMaxInv: ', inverseMaxSpeeds_original)
    # print('optimal speedMaxInv: ', inverseMaxSpeeds_.value)
    # print('\n')
    # print('original noWay: ', noWay_original)
    # print('optimal noWay: ', noWay_.value)
    # print('\n')
    # print('original areClosed: ', areClosed_original)
    # print('optimal areClosed: ', areClosed_.value)
    # print('\n')
    # print('original speedOrMaxSpeed: ', speedOrMaxSpeed_original)
    # print('optimal speedOrMaxSpeed: ', speedOrMaxSpeed_.value)
    
    
    # Creating the new Graph
    newGraph = nx.MultiDiGraph()
    
    for (i, j), index in edgeIndex.items():
        if speedOrMaxSpeed_.value[index] == 0:
            s = getInverse(inverseSpeeds_original[index])
        else:
            s = getInverse(inverseSpeeds_.value[index])
            
        if speedOrMaxSpeed_.value[index] == 1:
            ms = getInverse(inverseMaxSpeeds_original[index])
        else:
            ms = getInverse(inverseMaxSpeeds_.value[index])
            
        newGraph.add_edge(
                            i, j,
                            weight = np.nan, 
                            noWay = noWay_.value[index], 
                            isClosed = areClosed_.value[index], 
                            length = graph[i][j][0]['length'], 
                            speed = s,
                            maxSpeed = ms,
                            speedOrMaxSpeed = speedOrMaxSpeed_.value[index]
                        )
        
    updateGraphWeights(newGraph)
        
        
    sp = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    
    desiredPathWeight = getPathWeight(desiredPath, newGraph)
    optimalPathWeight = getPathWeight(sp, newGraph)
    print('\n')
    
    if (desiredPathWeight - optimalPathWeight) <= epsilon:
        print('The desired Path is equal to the Shortest Path')
    elif desiredPathWeight < optimalPathWeight:
        print('The desired Path is better than the Shortest Path')
    elif desiredPathWeight > optimalPathWeight:
        print('The desired Path is worse than the Shortest Path')
    
    
    print('Optimal Path Weight = ', optimalPathWeight)
    print('The path is: ', sp)
    # print('numbers after decimal point: ', len(str(optimalPathWeight).replace('.','')) - 1)
    print('\n')
    print('Desired Path Weight = ', desiredPathWeight)
    print('The path is: ', desiredPath)
    # print('numbers after decimal point: ', len(str(desiredPathWeight).replace('.','')) - 1)
        
    return newGraph


In [122]:
# Creating the test graph
G = nx.MultiDiGraph()
# G.add_edge(1, 2, weight=1, isNew=False)
G.add_edge(1, 2, weight=np.nan, noWay=0, isClosed=0, length=2, speed=30, maxSpeed=40, speedOrMaxSpeed=1)
G.add_edge(2, 6, weight=np.nan, noWay=0, isClosed=0, length=20, speed=10, maxSpeed=40, speedOrMaxSpeed=1)
G.add_edge(6, 7, weight=np.nan, noWay=0, isClosed=0, length=2, speed=30, maxSpeed=40, speedOrMaxSpeed=1)
G.add_edge(7, 8, weight=np.nan, noWay=0, isClosed=0, length=20, speed=60, maxSpeed=60, speedOrMaxSpeed=1)

G.add_edge(3, 1, weight=np.nan, noWay=0, isClosed=0, length=20, speed=15, maxSpeed=20, speedOrMaxSpeed=1)
G.add_edge(3, 4, weight=np.nan, noWay=0, isClosed=0, length=20, speed=7, maxSpeed=10, speedOrMaxSpeed=1)
G.add_edge(4, 5, weight=np.nan, noWay=0, isClosed=1, length=20, speed=5, maxSpeed=20, speedOrMaxSpeed=1)
G.add_edge(5, 8, weight=np.nan, noWay=0, isClosed=0, length=20, speed=19, maxSpeed=20, speedOrMaxSpeed=1)

desiredPath = [1, 3, 4, 5, 8]

# # Other  direction, for testing
#G.add_edge(2, 1, weight=np.nan, noWay=0, isClosed=0, length=20, speed=18, maxSpeed=20)
# G.add_edge(3, 1, weight=np.nan, noWay=0, isClosed=0, length=20, speed=15, maxSpeed=20)
# G.add_edge(4, 2, weight=np.nan, noWay=0, isClosed=0, length=20, speed=10, maxSpeed=20)
# G.add_edge(4, 3, weight=np.nan, noWay=0, isClosed=0, length=20, speed=7, maxSpeed=20)
# G.add_edge(5, 4, weight=np.nan, noWay=0, isClosed=1, length=20, speed=5, maxSpeed=20)

In [123]:
prepareGraph(G)
nx.shortest_path(G, source=desiredPath[0], target=desiredPath[-1], weight="weight")

[1, 2, 6, 7, 8]

In [124]:

new_graph = inverseShortestPath(G, desiredPath)


The optimal value is 20.794110275689224


The desired Path is equal to the Shortest Path
Optimal Path Weight =  2.4666666666666672
The path is:  [1, 2, 6, 7, 8]


Desired Path Weight =  2.466666666666668
The path is:  [1, 3, 4, 5, 8]


In [125]:
z = getGraphExplanation(G, new_graph, desiredPath)
explanationsPrinter(z)

These are the explanations why the desired path is nont the optimal path, and how it would be an optimal path:
   -   Edge (1, 3) is only one way (on the other side). If it was a two way edge, it would help make the path optimal.
   -   Edge (3, 4) has a current speed of 7 (heavy traffic). If it had a current speed of 9.677419354838703 (less traffic), it would help make the path optimal.
   -   Edge (4, 5) is closed. If it was open, it would help make the path optimal.
   -   Edge (5, 8) has a maximim speed of 20. If it had a maximum speed of 49.99999999999999, it would help make the path optimal.
